#**DEEP LEARNING Modeling**

###**GRU**

In [ ]:
# Importing necessary libraries

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

from tensorflow import keras
from tensorflow.keras.metrics import AUC
import tensorflow_datasets as tfds

from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Bidirectional, LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Bidirectional, LSTM

from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

from keras.layers import Bidirectional, LSTM, GRU
import keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/Design Project"

/content/drive/My Drive/Design Project


In [ ]:
ls

 best_model2.h5                                                  LSTM_model2.h5
 best_tfidf_xgboost_model.pkl                                    LSTM_model.h5
 bow_tuned_lgb_model.pkl                                         model_dir/
 cnn_model.h5                                                    new_tuned_xgboost_model.pkl
'Design Project - ** 1. Pre-Processing.ipynb'                    preprocessed_test_data.csv
'Design Project - ** 2. Feature Engineering   Embedding.ipynb'   preprocessed_train_data.csv
 Design_Project_Code_2_for_submission_ipynb.ipynb                preprocessed_valid_data.csv
'google news vectors'                                            proppy_1.0.dev.tsv
 GoogleNews-vectors-negative300.bin.gz                           proppy_1.0.test.tsv
 google_w2v_tuned_lightgbm_model.pkl                             proppy_1.0.train.tsv
 googlew2v_tuned_xgboost_model.pkl                               tfidf_tuned_lgb_model.pkl
 GRU_model.h5                                   

In [ ]:
train_data = pd.read_csv('preprocessed_train_data.csv')
test_data = pd.read_csv('preprocessed_test_data.csv')
valid_data = pd.read_csv('preprocessed_valid_data.csv')

In [ ]:
# Selecting the features and target variables
X = train_data["text_preprocessed"]
y = train_data["propaganda_label"]

In [ ]:
# Train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50, stratify=y)

In [ ]:
# Adding parameter values
num_words = 5000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [ ]:
# Tokenize our training data
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(X_train)

# Get max training sequence length
# maxlen = max([len(x) for x in train_sequences])
maxlen = 1000

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
print("Word index:\n", {key: word_index[key] for key in list(word_index)[:3]})
print("\nTraining sequences:\n", train_sequences[:3])
print("\nPadded training sequences:\n", train_padded[:3])
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Word index:
 {'<UNK>': 1, 'said': 2, 'year': 3}

Training sequences:
 [[56, 4421, 270, 84, 25, 1696, 815, 1, 1331, 4694, 1, 304, 2818, 1, 523, 509, 1, 1, 1, 2286, 23, 71, 383, 842, 1288, 97, 902, 2867, 378, 108, 67, 523, 2286, 214, 718, 50, 1, 611, 241, 10, 1, 29, 2, 2286, 45, 718, 303, 582, 15, 353, 230, 1, 1, 1, 25, 2, 71, 1148, 378, 97, 207, 464, 1, 1389, 152, 1084, 3254, 352, 1, 3235, 2451, 1, 1285, 583, 523, 319, 1, 25, 601, 1, 1, 1, 26, 471, 810, 25, 2031, 1, 608, 16, 738, 495, 97, 214, 523, 504, 523, 16, 72, 1, 1, 1157, 236, 1, 1, 2, 16, 738, 3490, 16, 482, 150, 527, 25, 2, 264, 2698, 363, 1942, 329, 1106, 1724, 3984, 329, 492, 363, 1106, 1586, 1378, 2045, 323, 1942, 93, 324, 2146, 2298, 1, 269, 371, 823, 1106, 632, 172, 632, 1378, 329, 492, 253, 223, 1615, 416, 2428, 2278, 464, 6, 2530, 34, 1, 1, 1, 1, 1, 1, 318, 4, 6, 99, 409, 516, 1633, 2663, 685, 464, 6, 1, 2286, 464, 6, 568, 1059, 97, 543, 1148, 1, 36, 141, 251, 1, 1113, 1112, 2987, 2206, 370, 207, 56, 538, 25, 38, 2217, 72

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

print("Testing sequences:\n", test_sequences[:3])
print("\nPadded testing sequences:\n", test_padded[:3])
print("\nPadded testing shape:",test_padded.shape)

Testing sequences:
 [[547, 3934, 211, 2906, 240, 305, 2442, 1, 3744, 4273, 567, 110, 39, 1771, 547, 3934, 6, 848, 601, 3470, 1, 18, 2716, 331, 1, 2325, 432, 331, 676, 158, 704, 4200, 3250, 851, 2555, 388, 200, 47, 2170, 230, 599, 209, 214, 119, 114, 217, 331, 98, 800, 1164, 1, 3419, 773, 14, 1, 18, 331, 148, 1455, 56, 39, 1, 18, 848, 1, 211, 2688, 659, 941, 1], [113, 281, 42, 1, 297, 1, 1, 123, 177, 2463, 443, 29, 432, 737, 872, 168, 4, 14, 4318, 1049, 1638, 768, 102, 256, 2463, 1972, 434, 281, 443, 624, 4145, 1124, 207, 102, 109, 2463, 2707, 1380, 304, 604, 2333, 64, 6, 1, 554, 1638, 768, 2463, 1972, 66, 11, 169, 5, 281, 1, 1, 1, 1, 58, 2972, 1194, 168, 124, 1, 9, 3869, 1, 1, 238, 2463, 1, 147, 1, 434, 281, 443, 624, 432, 225, 974, 296, 556, 1, 2162, 109, 544, 246, 1967, 42, 1, 297, 1, 3028, 115, 1, 281, 1773, 2463, 4843, 31, 1, 123, 225, 1, 13, 3, 54, 2085, 1434, 238, 336, 29, 177, 2463, 443, 69, 1, 562, 1, 2768, 1, 1, 2463, 1147, 1, 1, 1, 109, 3858, 2463, 711, 764, 562, 2043, 2962, 

In [ ]:
train_labels = np.array(y_train)
test_labels = np.array(y_test)

In [ ]:
# Recode class labels from 1 and -1 to 0 and 1

train_labels[train_labels == -1] = 0
test_labels[test_labels == -1] = 0

In [ ]:
from sklearn.utils import class_weight

# Get class weights from training data
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes =np.unique(train_labels), y= train_labels)

# Create a dictionary to map class index to weight
class_weights_dict = dict(enumerate(class_weights))

print(class_weights_dict)

{0: 0.5629032888819365, 1: 4.4743549891202985}


**Model Building**

In [ ]:
%%time

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 256),

    # SpatialDropout1D(0.2),
    GRU(128, return_sequences = True),
    # Dropout(0.2),

    keras.layers.Dense(1, activation='sigmoid'),

    GRU(128, return_sequences=False),
    # Dropout(0.2),

    keras.layers.Dense(64, activation='relu'),  # Additional dense layer
    Dropout(0.1),  # Dropout layer after the dense layer

    keras.layers.Dense(1, activation='sigmoid')

])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", get_f1, AUC()])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

checkpoint = keras.callbacks.ModelCheckpoint("GRU_model.h5", save_best_only=True)

history = model.fit(train_padded, train_labels, epochs=5, validation_data=(test_padded, test_labels), callbacks=[early_stop, checkpoint])


Epoch 1/5


Cause: Unable to locate the source code of <function get_f1 at 0x7cf7233f0f70>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function get_f1 at 0x7cf7233f0f70>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
900/900 [==============================] - 2536s 3s/step - loss: 0.3573 - accuracy: 0.8883 - get_f1: 0.0000e+00 - auc_2: 0.5044 - val_loss: 0.3516 - val_accuracy: 0.8883 - val_get_f1: 0.0000e+00 - val_auc_2: 0.5002
Epoch 2/5
900/900 [==============================] - 2534s 3s/step - loss: 0.3526 - accuracy: 0.8883 - get_f1: 0.0000e+00 - auc_2: 0.4959 - val_loss: 0.3503 - val_accuracy: 0.8883 - val_get_f1: 0.0000e+00 - val_auc_2: 0.5002
Epoch 3/5
900/900 [==========================

In [ ]:
# Saving the model to an HDF5 file

model.save("GRU_model.h5")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-f46e08e4de11>", line 3, in <cell line: 3>
    model.save("GRU_model.h5")
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 237, in make_fid
    fid = h5f.create(name, h5f.ACC_TRUNC, fapl=fapl, fcpl=fcpl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 126, in h5py.h5f.create
OSError: [Errno 107] Unable to crea

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model_gru = load_model("GRU_model.h5")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-2eb24c23102e>", line 4, in <cell line: 4>
    model_gru = load_model("GRU_model.h5")
  File "/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py", line 262, in load_model
    return legacy_sm_saving_lib.load_model(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/src/saving/legacy/save.py", line 234, in load_model
    raise IOError(
OSError: No file or directory found at GRU_model.h5

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showt

In [ ]:
model_gru

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-45bf963c15ac>", line 1, in <cell line: 1>
    model_gru
NameError: name 'model_gru' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py"

In [ ]:
# Make predictions on the training data using the model

predictions = model_gru.predict(test_padded)
predicted_labels = np.round(predictions)

# Recode class labels from 1 and 0 to -1 and 1
test_labels[test_labels == 0] = -1
predicted_labels[predicted_labels == 0] = -1

# Generate the confusion matrix
confusion_mat = confusion_matrix(test_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)

# Computing and plotting the Confusion matrix
cf_matrix = confusion_matrix(test_labels, predicted_labels)

categories  = ['Negative','Positive']
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

plt.figure(figsize=(8, 6)) # Create a new figure for confusion matrix
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

# Generate the classification report
class_report = classification_report(test_labels, predicted_labels)
print("Classification Report:\n", class_report)

# Calculate and print the ROC AUC score
auc_score = roc_auc_score(test_labels, predictions)
print("ROC AUC Score:", auc_score)

# Generate ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, predictions)
sns.set_style("darkgrid")
plt.figure()
plt.plot(fpr, tpr, linestyle='--',color='orange',label=f'Model (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--' , color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-eeefe5148844>", line 3, in <cell line: 3>
    predictions = model_gru.predict(test_padded)
NameError: name 'model_gru' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dis

In [ ]:
# Make predictions on the Validation data using the model

test_sequences = tokenizer.texts_to_sequences(valid_data.text_preprocessed)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

predictions = model_gru.predict(test_padded)
predicted_labels = np.round(predictions)

# Changing the label name
y_test = valid_data.propaganda_label
test_labels = np.array(y_test)

# Recode class labels from 1 and 0 to -1 and 1
test_labels[test_labels == 0] = -1
predicted_labels[predicted_labels == 0] = -1

# Generate the confusion matrix
confusion_mat = confusion_matrix(test_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)

# Computing and plotting the Confusion matrix
cf_matrix = confusion_matrix(test_labels, predicted_labels)

categories  = ['Negative','Positive']
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

plt.figure(figsize=(8, 6)) # Create a new figure for confusion matrix
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

# Generate the classification report
class_report = classification_report(test_labels, predicted_labels)
print("Classification Report:\n", class_report)

# Calculate and print the ROC AUC score
auc_score = roc_auc_score(test_labels, predictions)
print("ROC AUC Score:", auc_score)

# Generate ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, predictions)
sns.set_style("darkgrid")
plt.figure()
plt.plot(fpr, tpr, linestyle='--',color='orange',label=f'Model (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--' , color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Make predictions on the Test data using the model

test_sequences = tokenizer.texts_to_sequences(test_data.text_preprocessed)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

predictions = model_gru.predict(test_padded)
predicted_labels = np.round(predictions)

# Changing the label name
y_test = test_data.propaganda_label
test_labels = np.array(y_test)

# Recode class labels from 1 and 0 to -1 and 1
test_labels[test_labels == 0] = -1
predicted_labels[predicted_labels == 0] = -1

# Generate the confusion matrix
confusion_mat = confusion_matrix(test_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)

# Computing and plotting the Confusion matrix
cf_matrix = confusion_matrix(test_labels, predicted_labels)

categories  = ['Negative','Positive']
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

plt.figure(figsize=(8, 6)) # Create a new figure for confusion matrix
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

# Generate the classification report
class_report = classification_report(test_labels, predicted_labels)
print("Classification Report:\n", class_report)

# Calculate and print the ROC AUC score
auc_score = roc_auc_score(test_labels, predictions)
print("ROC AUC Score:", auc_score)

# Generate ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, predictions)
sns.set_style("darkgrid")
plt.figure()
plt.plot(fpr, tpr, linestyle='--',color='orange',label=f'Model (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--' , color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()